# Emotions Text Classifier project

### Project Prerequisites

In [26]:
import pandas as pd
import numpy as np
import spacy
import joblib
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from transformers import RobertaTokenizerFast, TFRobertaForSequenceClassification, pipeline

In [2]:
plt.style.use("default")
plt.style.use("seaborn-darkgrid")

### Preparing the Data

In [3]:
df = pd.read_csv("Data/emotion-labels-train.csv")
df.head()
text_df = pd.read_csv("Data/processed_data.csv")
label_df = pd.read_csv("Data/processed_labels.csv")

In [4]:
test_df = pd.read_csv("Data/emotion-labels-test.csv")
test_df.head()

,text,label
0,You must be knowing #blithe means (adj.) Happ...,joy
1,Old saying 'A #smile shared is one gained for ...,joy
2,Bridget Jones' Baby was bloody hilarious 😅 #Br...,joy
3,@Elaminova sparkling water makes your life spa...,joy
4,I'm tired of everybody telling me to chill out...,joy


In [5]:
test_x = test_df["text"]
test_y = test_df["label"]

#### Checking for NaN's